In [44]:

import numpy as np
from random import random


class MLP(object):
    """A Multilayer Perceptron class.
    """

    def __init__(self, num_inputs=3, hidden_layers=[3, 3], num_outputs=2):
        """Constructor for the MLP. Takes the number of inputs,
            a variable number of hidden layers, and number of outputs

        Arguments:
            num_inputs (int): Number of inputs
            hidden_layers (list): A list of ints for the hidden layers
            num_outputs (int): Number of outputs
        """

        self.num_inputs = num_inputs
        self.hidden_layers = hidden_layers
        self.num_outputs = num_outputs

        # to simplify the representation of the layers
        layers = [num_inputs] + hidden_layers + [num_outputs]

        # create random connection weights for the layers, segments the "layer" array into the necessary matrices.
        weights = []
        for i in range(len(layers) - 1):
            w = np.random.rand(layers[i], layers[i + 1])
            weights.append(w)
        self.weights = weights

        # save derivatives per layer
        derivatives = []
        for i in range(len(layers) - 1):
            d = np.zeros((layers[i], layers[i + 1]))
            derivatives.append(d)
        self.derivatives = derivatives

        # save activations per layer
        activations = []
        for i in range(len(layers)):
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations


    def forward_propagate(self, inputs):
        """Computes forward propagation of the network based on input signals.

        Args:
            inputs (ndarray): Input signals
        Returns:
            activations (ndarray): Output values
        """

        # the input layer activation is just the input itself
        activations = inputs

        # save the activations for backpropogation
        self.activations[0] = activations

        # iterate through the network layers
        for i, w in enumerate(self.weights):
            # calculate matrix multiplication between previous activation and weight matrix
            net_inputs = np.dot(activations, w)

            # apply sigmoid activation function
            activations = self._sigmoid(net_inputs)

            # save the activations for backpropogation
            self.activations[i + 1] = activations

        # return output layer activation
        return activations


    def back_propagate(self, error):
        """Backpropogates an error signal.
        Args:
            error (ndarray): The error to backprop.
        Returns:
            error (ndarray): The final error of the input
        """

        # iterate backwards through the network layers
        for i in reversed(range(len(self.derivatives))):

            # get activation for previous layer
            activations = self.activations[i+1]

            # apply sigmoid derivative function
            delta = error * self._sigmoid_derivative(activations)

            # reshape delta as to have it as a 2d array
            delta_re = delta.reshape(delta.shape[0], -1).T

            # get activations for current layer
            current_activations = self.activations[i]

            # reshape activations as to have them as a 2d column matrix
            current_activations = current_activations.reshape(current_activations.shape[0],-1)

            # save derivative after applying matrix multiplication
            self.derivatives[i] = np.dot(current_activations, delta_re)

            # backpropogate the next error
            error = np.dot(delta, self.weights[i].T)


    def train(self, inputs, targets, epochs, learning_rate):
        """Trains model running forward prop and backprop
        Args:
            inputs (ndarray): X
            targets (ndarray): Y
            epochs (int): Num. epochs we want to train the network for
            learning_rate (float): Step to apply to gradient descent
        """
        # now enter the training loop
        for i in range(epochs):
            sum_errors = 0

            # iterate through all the training data
            for j, input in enumerate(inputs):
                target = targets[j]

                # activate the network!
                output = self.forward_propagate(input)

                error = target - output

                self.back_propagate(error)

                # now perform gradient descent on the derivatives
                # (this will update the weights
                self.gradient_descent(learning_rate)

                # keep track of the MSE for reporting later
                sum_errors += self._mse(target, output)

            # Epoch complete, report the training error
            print("Error: {} at epoch {}".format(sum_errors / len(items), i+1))

        print("Training complete!")
        print("=====")


    def gradient_descent(self, learningRate=1):
        """Learns by descending the gradient
        Args:
            learningRate (float): How fast to learn.
        """
        # update the weights by stepping down the gradient
        for i in range(len(self.weights)):
            weights = self.weights[i]
            derivatives = self.derivatives[i]
            weights += derivatives * learningRate


    def _sigmoid(self, x):
        """Sigmoid activation function
        Args:
            x (float): Value to be processed
        Returns:
            y (float): Output
        """

        y = 1.0 / (1 + np.exp(-x))
        return y


    def _sigmoid_derivative(self, x):
        """Sigmoid derivative function
        Args:
            x (float): Value to be processed
        Returns:
            y (float): Output
        """
        return x * (1.0 - x)


    def _mse(self, target, output):
        """Mean Squared Error loss function
        Args:
            target (ndarray): The ground trut
            output (ndarray): The predicted values
        Returns:
            (float): Output
        """
        return np.average((target - output) ** 2)

In [45]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Votre code MLP ici
# (coller votre classe MLP ici)

# Étape 1 : Charger et préparer les données
iris = datasets.load_iris()

# Charger les caractéristiques (features) et les étiquettes (labels)
X = iris.data  # Caractéristiques
y = iris.target.reshape(-1, 1)  # Labels

# Normaliser les caractéristiques (X) entre 0 et 1
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Appliquer one-hot encoding aux labels
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

# Diviser le jeu de données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2)

# Étape 2 : Créer et entraîner le MLP
mlp = MLP(num_inputs=4, hidden_layers=[5], num_outputs=3)  # 4 entrées, 1 couche cachée avec 5 neurones, 3 sorties
mlp.train(X_train, y_train, epochs=100, learning_rate=0.1)

# Étape 3 : Tester le MLP sur les données de test
correct = 0
for i, input in enumerate(X_test):
    # Faire une prédiction
    output = mlp.forward_propagate(input)
    
    # Trouver l'index de la classe prédite (avec la plus grande probabilité)
    predicted_class = np.argmax(output)
    
    # Trouver l'index de la vraie classe
    true_class = np.argmax(y_test[i])
    
    if predicted_class == true_class:
        correct += 1

# Afficher les résultats
print()
print("Précision sur l'ensemble de test: {:.2f}%".format((correct / len(X_test)) * 100))

[array([0., 0., 0., 0.]), array([0., 0., 0., 0., 0.]), array([0., 0., 0.])]
Error: 0.04131805201407656 at epoch 1
Error: 0.027360095010529224 at epoch 2
Error: 0.0265835739394825 at epoch 3
Error: 0.026471570196743864 at epoch 4
Error: 0.02636625702469146 at epoch 5
Error: 0.02623662627380885 at epoch 6
Error: 0.026073380901251483 at epoch 7
Error: 0.025866513182753546 at epoch 8
Error: 0.025604058581643722 at epoch 9
Error: 0.025272511981196677 at epoch 10
Error: 0.024858383797652338 at epoch 11
Error: 0.024351195920684333 at epoch 12
Error: 0.023747545497484302 at epoch 13
Error: 0.023054692656852577 at epoch 14
Error: 0.02229161858718809 at epoch 15
Error: 0.021486621565717404 at epoch 16
Error: 0.02067237906359426 at epoch 17
Error: 0.019880336106421405 at epoch 18
Error: 0.01913609136369977 at epoch 19
Error: 0.018456826783808222 at epoch 20
Error: 0.01785101913238143 at epoch 21
Error: 0.01731987663458143 at epoch 22
Error: 0.0168595526008531 at epoch 23
Error: 0.0164633159836841

In [36]:
from random import randint
nb = randint(1,29)
print(X_test[nb], y_test[nb])
output = mlp.forward_propagate(X_test[nb])
predicted_class = np.argmax(output)
true_class = np.argmax(y_test[nb])
print("Class prédite", predicted_class, " vrai classe", true_class)


[0.38888889 1.         0.08474576 0.125     ] [1. 0. 0.]
Class prédite 0  vrai classe 0
